# TinyVGG 파일형식
- model.eval 을 진행하기 위해선 다음과 같은 파일 형식을 만들어야 합니다. 해당 구조를 만들기 위한 코드는 검증용 이미지 폴더 전처리용 코드 move_images_based_on_defect 코드에 있습니다. 코드를 따라 진행하시면 됩니다. (클릭하시면 디렉토리가 제대로 표시됩니다.)
|-- data
    |-- val
        |-- normal
            |-- 20240806_170349.png
            |-- 20240806_170404.png ...
        |-- defect
            |-- 20240806_170360.png
            |-- 20240806_170467.png ...

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import torch
from torch import nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from sklearn.metrics import classification_report, confusion_matrix
import numpy as np
import os
import shutil

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

# 검증용 이미지 폴더 전처리용 코드 
### img_directory에 검증용 이미지 폴더(jpg 형식) 경로를 입력하신 후, txt_directory에는 검증용 이미지 라벨(txt 형식) 경로를 입력해 주십시오. 라벨에 따라서 오류와 정상 이미지를 각각의 폴더로 분류할 것입니다. 제공된 데이터 형식에 따라, 라벨 txt데이터에 아무것도 적혀있지 않은 데이터를 정상 데이터로 분류합니다.

In [ ]:
img_directory = 'printing/images' # TODO: 검증용 이미지 디렉토리 경로를 입력해 주십시오.
txt_directory = 'printing/labels' # TODO: 검증용 이미지 라벨 디렉토리 경로를 입력해 주심시오.
normal_directory = 'val/normal' # 검증용 정상 이미지 저장 경로
defect_directory = 'val/defect' # 검증용 비정상 이미지 저장 경로

def move_images_based_on_defect(img_dir, txt_dir, normal_dir, defect_dir):
    os.makedirs(normal_dir, exist_ok=True)
    os.makedirs(defect_dir, exist_ok=True)

    for img_file in os.listdir(img_dir):
        if img_file.endswith('.jpg'):
            img_path = os.path.join(img_dir, img_file)
            txt_path = os.path.join(txt_dir, img_file.replace('.jpg', '.txt'))

            with open(txt_path, 'r') as f:
                text_content = f.read().strip()

            if text_content == "":
                shutil.move(img_path, os.path.join(normal_dir, img_file))
                print(f"Moved {img_file} to {normal_dir}")
            else:
                shutil.move(img_path, os.path.join(defect_dir, img_file))
                print(f"Moved {img_file} to {defect_dir}")

move_images_based_on_defect(img_directory, txt_directory, normal_directory, defect_directory)

### 최종적으로 평가를 진행할 테스트 데이터셋에 기본적인 전처리를 진행하여 테스트 데이터셋을 로드합니다. test_dir에 검증용 데이터 경로(val)를 입력해주어야 합니다. (val 디렉토리 밑에는 defect와 normal이 있습니다.)

In [ ]:
test_dir = "/content/drive/MyDrive/printing/test"

manual_transforms = transforms.Compose([
    transforms.Resize((2532, 824)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

print("Loading TEST dataset...")
test_dataset = datasets.ImageFolder(root=test_dir, transform=manual_transforms)
test_dataloader = DataLoader(test_dataset, batch_size=16, shuffle=False)
class_names = test_dataset.classes
print(f"Class names: {class_names}")
print(f"Loaded {len(test_dataset)} test images.")

### 모델의 평가를 진행하는 코드입니다.

In [ ]:
def evaluate_model(model, dataloader, device, class_names):
    model.eval()
    true_labels, predicted_labels = [], []
    with torch.no_grad():
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            true_labels.extend(labels.cpu().numpy())
            predicted_labels.extend(preds.cpu().numpy())

    print("Confusion Matrix:")
    print(confusion_matrix(true_labels, predicted_labels))
    print("\nClassification Report:")
    print(classification_report(true_labels, predicted_labels, target_names=class_names))


### 학습에 활용한 TinyVGG 모델 구조와 동일하게 모델 구조를 정의하였습니다.

In [ ]:
class TinyVGG(nn.Module):

    def __init__(self, input_shape: int, hidden_units: int, output_shape: int) -> None:
        super().__init__()
        self.conv_block_1 = nn.Sequential(
            nn.Conv2d(in_channels=input_shape,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.conv_block_2 = nn.Sequential(
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.Conv2d(in_channels=hidden_units,
                      out_channels=hidden_units,
                      kernel_size=3,
                      stride=1,
                      padding=0),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(in_features=hidden_units * 127890, out_features=output_shape)
        )

    def forward(self, x):
        x = self.conv_block_1(x)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        return x


### 학습 과정에서 저장된 최적의 가중치 tinyVGG_weights_T.pkl를 로드하여, 최종 테스트 데이터셋에 대한 평가를 진행합니다.

In [ ]:
tinyvgg_model = TinyVGG(input_shape=3, hidden_units=10, output_shape=2)
tinyvgg_model.load_state_dict(torch.load("/content/drive/MyDrive/models/tinyVGG_weights_T.pkl", map_location=device))
tinyvgg_model = tinyvgg_model.to(device)
tinyvgg_model.eval()

print("Evaluating TinyVGG on TEST dataset...")
evaluate_model(tinyvgg_model, test_dataloader, device, class_names)
